# 점수 예측 분포도 그려보기

dev.csv에 들어있는 label 결과를 모델이 얼마나 잘 예측하고 있는 지를 측정하기 위한 코드를 작성하고, 가장 점수 차이가 많이 나는 상위 20개의 결과물을 csv로 내보내고, 히스토그램 및 산점도를 그려보기로 했습니다. 

In [ ]:
import torch
import pandas as pd
import transformers
import matplotlib.pyplot as plt 
import pytorch_lightning as pl

# 1. Model 클래스 정의
class Model(pl.LightningModule):
    def __init__(self, model_name, lr):
    
		    # pl.LightningModule 초기화
        super().__init__()
        
	      # 모델명(model_name) 및 plm(pretrained learning model)을 불러옵니다. 
	      # transformers.AutoModelForSequenceClassification를 사용 
	      # num_labels=1 -> 예측해야 할 값이 하나 (유사도 점수)
        self.model_name = model_name
        self.plm = transformers.AutoModelForSequenceClassification.from_pretrained(
            pretrained_model_name_or_path=model_name, num_labels=1
        )

		# forward: 입력값 (input_ids, attention_mask) -> plm -> logits 반환
    def forward(self, input_ids, attention_mask=None):
        outputs = self.plm(input_ids=input_ids, attention_mask=attention_mask)
        return outputs['logits']

# 2. dev.csv 데이터 로드
dev_data = pd.read_csv('../data/dev.csv')

# 3. 모델 로드 및 평가 모드 전환
model = torch.load('model_large_20_msk.pt')
model = torch.load('model_large_20.pt')
model.eval()

# 4. 토크나이저 준비
tokenizer = transformers.AutoTokenizer.from_pretrained('klue/roberta-small')

# 5. 데이터 전처리 (dev.csv 텍스트를 토큰화)
# 두 문장을 '[SEP]' 토큰으로 연결
# return_tensors='pt' -> 토큰화된 데이터를 모델에 입력할 수 있도록 함
def preprocess_data(data, tokenizer):
    texts = ['[SEP]'.join([item['sentence_1'], item['sentence_2']]) for _, item in data.iterrows()]
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=160)
    return inputs

# 결과값을 inputs에 넣음
inputs = preprocess_data(dev_data, tokenizer)

# 6. 모델 예측 수행 
# torch.no_grad(): 모델 -> 예측값 계산, 역전파 비활성화
with torch.no_grad():
    outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
    predictions = outputs.squeeze().numpy()

# 7. 실제 값 불러오기
actuals = dev_data['label'].values

# 8. 예측값과 실제값 비교 (차이 계산)
differences = abs(actuals - predictions)

# 9. 차이가 큰 데이터 상위 20개 출력
dev_data['predicted'] = predictions
dev_data['difference'] = differences
top_differences = dev_data.nlargest(20, 'difference')

# 차이가 큰 20개의 데이터를 csv 파일 형태로 저장
print("차이가 큰 상위 20개 데이터:\n", top_differences[['sentence_1', 'sentence_2', 'label', 'predicted', 'difference']].to_csv('difference_without_mask.csv', index=False))

# 10. 예측값과 실제값 비교 시각화 (히스토그램)
plt.figure(figsize=(10,6))
plt.hist(actuals, bins=20, alpha=0.5, label='Actuals', color='blue', edgecolor='black')
plt.hist(predictions, bins=20, alpha=0.5, label='Predictions', color='red', edgecolor='black')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.title('Actual vs Predicted Similarity Score Distribution')
plt.legend()
plt.show()

# 11. 예측값과 실제값 차이 시각화 (Scatter Plot)
plt.figure(figsize=(10,6))
plt.scatter(actuals, predictions, alpha=0.5)
plt.plot([0, 5], [0, 5], color='red', linestyle='--')  # 완벽히 일치하는 경우
plt.xlabel('Actual Similarity')
plt.ylabel('Predicted Similarity')
plt.title('Actual vs Predicted Similarity (STS)')
plt.grid(True)
plt.show()

# 결과물 (그래프) 
### 마스킹을 적용한 모델  

![model_large_20_msk.pt](images/model_large_20_msk_score_distribution_graph.png)

### 마스킹을 적용하지 않은 모델 (히스토그램)

![model_large_20_msk.pt](images/model_large_20_score_distribution_graph.png)

### 마스킹을 적용하지 않은 모델 (산점도)

![scatter_plot.pt](images/scatter_plot.png)